# Surrounding Venues for Each Arcade

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Loading Arcade DataFrame](#Loading-Arcade-DataFrame)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Making Surrounding Venues DataFrame](#Making-Surrounding-Venues-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Arcade and Distance](#Sorting-by-Arcade-and-Distance)

[Ensuring Unique Venues](#Ensuring-Unique-Venues)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Loading Arcade DataFrame

In [2]:
kc_arcades = pd.read_csv('Arcade_final_clean.csv')
kc_arcades = kc_arcades.drop('Unnamed: 0', axis = 1)
print(len(kc_arcades))
kc_arcades

7


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space,Not Rated
1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop,Not Rated
2,Chaumiere,4fbefafae4b02c8542b9389a,Arcade Xbox,1305,No Address,39.162169,-94.546515,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Library District,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,204,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza,Not Rated
4,Winnetonka,51b790b2498e9e3b028cfaef,Joe Cool Arcade,1834,No Address,39.173578,-94.489309,4bf58dd8d48988d182941735,Theme Park,Not Rated
5,Walnut Grove,54076348498ed3ee81d32983,Scarritt Arcade,128,819 Walnut St,39.103793,-94.581555,4bf58dd8d48988d130941735,Building,Not Rated
6,Quality Hill,5ad817f3c666667613ddcb70,Broadway Arcade,152,1000 Broadway Blvd Ste 200,39.102272,-94.588670,4bf58dd8d48988d1e1931735,Arcade,Not Rated


### Connecting to FourSquare

In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 20

### Making Surrounding Venues DataFrame

In [5]:
#assign variables for the search
search_query = ''
radius = 50
print(search_query + ' .... OK!')

 .... OK!


In [6]:
surrounding_venues = pd.DataFrame(columns = ['Arcade','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
surrounding_venues

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [7]:
for index, arcade in enumerate(kc_arcades['Venue_ID']):
    try:
        arcade_name = kc_arcades['Venue_Name'][index]
        latitude = kc_arcades['Venue_Latitude'][index]
        longitude = kc_arcades['Venue_Longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        print('No response')
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
            continue
        try:
            surrounding_venues = surrounding_venues.append({'Venue_Distance':ven_dist,'Arcade':arcade_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)
        except:
            surrounding_venues = surrounding_venues.append({'Arcade': 'Null'}, ignore_index=True)

In [8]:
print(len(surrounding_venues))
surrounding_venues.head()

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Sean's Arcade And Mini Bar,4bff06fe10899c74d66215c5,Sarah Colman-Livengood Park,401,59th & Lane,39.015510,-94.475734,4bf58dd8d48988d1e7941735,Playground
1,Sean's Arcade And Mini Bar,4bd06bc49854d13a49a1f74d,Laurel Hills Elementary,878,5401 Lane,39.025665,-94.475555,4f4533804b9074f6e4fb0105,Elementary School
2,Sean's Arcade And Mini Bar,4dce8ec78877851243fa0958,Graceway Ball Fields,577,No Address,39.021284,-94.484193,4bf58dd8d48988d1e8941735,Baseball Field
3,Sean's Arcade And Mini Bar,4e9cca65dab4aec571ae9d98,Shays Animal Kingdom,14,No Address,39.018282,-94.478711,4bf58dd8d48988d1e5941735,Dog Run
4,Sean's Arcade And Mini Bar,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,0,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space


In [9]:
surrounding_venues.to_csv('surrounding_raw.csv')

### Dropping the Nulls

In [10]:
surrounding_venues = pd.read_csv('surrounding_raw.csv')
surrounding_venues = surrounding_venues.drop('Unnamed: 0', axis= 1)
print(len(surrounding_venues))
surrounding_venues.head()

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Sean's Arcade And Mini Bar,4bff06fe10899c74d66215c5,Sarah Colman-Livengood Park,401,59th & Lane,39.015510,-94.475734,4bf58dd8d48988d1e7941735,Playground
1,Sean's Arcade And Mini Bar,4bd06bc49854d13a49a1f74d,Laurel Hills Elementary,878,5401 Lane,39.025665,-94.475555,4f4533804b9074f6e4fb0105,Elementary School
2,Sean's Arcade And Mini Bar,4dce8ec78877851243fa0958,Graceway Ball Fields,577,No Address,39.021284,-94.484193,4bf58dd8d48988d1e8941735,Baseball Field
3,Sean's Arcade And Mini Bar,4e9cca65dab4aec571ae9d98,Shays Animal Kingdom,14,No Address,39.018282,-94.478710,4bf58dd8d48988d1e5941735,Dog Run
4,Sean's Arcade And Mini Bar,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,0,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space


In [11]:
surrounding_venues = surrounding_venues[surrounding_venues != 'Null']
surrounding_venues = surrounding_venues[surrounding_venues != '[]']
surrounding_venues = surrounding_venues.dropna()
print(len(surrounding_venues))
surrounding_venues = surrounding_venues.reset_index()
surrounding_venues = surrounding_venues.drop(['index'], axis = 1)
surrounding_venues.head(20)

140


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Sean's Arcade And Mini Bar,4bff06fe10899c74d66215c5,Sarah Colman-Livengood Park,401,59th & Lane,39.015510,-94.475734,4bf58dd8d48988d1e7941735,Playground
1,Sean's Arcade And Mini Bar,4bd06bc49854d13a49a1f74d,Laurel Hills Elementary,878,5401 Lane,39.025665,-94.475555,4f4533804b9074f6e4fb0105,Elementary School
2,Sean's Arcade And Mini Bar,4dce8ec78877851243fa0958,Graceway Ball Fields,577,No Address,39.021284,-94.484193,4bf58dd8d48988d1e8941735,Baseball Field
3,Sean's Arcade And Mini Bar,4e9cca65dab4aec571ae9d98,Shays Animal Kingdom,14,No Address,39.018282,-94.478710,4bf58dd8d48988d1e5941735,Dog Run
4,Sean's Arcade And Mini Bar,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,0,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space
5,Sean's Arcade And Mini Bar,5f29ce36dada2c4da2b70b12,Mount Carmel Cemetary,1115,No Address,39.023317,-94.467750,4bf58dd8d48988d15c941735,Cemetery
6,Sean's Arcade And Mini Bar,4e31c42f2fb6ede816e33fa5,Redbox,1020,8700 E 63rd St,39.009638,-94.483069,4bf58dd8d48988d126951735,Video Store
7,Sean's Arcade And Mini Bar,591ef4299ba3e53785aacba9,Shell,1198,8435 E 63rd St,39.009499,-94.486988,4bf58dd8d48988d113951735,Gas Station
8,Sean's Arcade And Mini Bar,4e4806e762e148603b87a23f,Graceway,757,5460 Blue Ridge Cut Off,39.024537,-94.482008,4bf58dd8d48988d132941735,Church
9,Sean's Arcade And Mini Bar,50de6836e4b09e22ed1b7185,quicktrip 63rd st and blue ridge cutoff,1053,No Address,39.009168,-94.482493,4d954b0ea243a5684a65b473,Convenience Store


In [12]:
count = -1
distance_drop_list = list()
for dist in surrounding_venues['Venue_Distance']:
    count = count + 1
    if dist == 0:
        distance_drop_list.append(count)
print(len(distance_drop_list))

9


In [13]:
surrounding_venues = surrounding_venues.drop(index = distance_drop_list)
print(len(surrounding_venues))
surrounding_venues.head()

131


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Sean's Arcade And Mini Bar,4bff06fe10899c74d66215c5,Sarah Colman-Livengood Park,401,59th & Lane,39.015510,-94.475734,4bf58dd8d48988d1e7941735,Playground
1,Sean's Arcade And Mini Bar,4bd06bc49854d13a49a1f74d,Laurel Hills Elementary,878,5401 Lane,39.025665,-94.475555,4f4533804b9074f6e4fb0105,Elementary School
2,Sean's Arcade And Mini Bar,4dce8ec78877851243fa0958,Graceway Ball Fields,577,No Address,39.021284,-94.484193,4bf58dd8d48988d1e8941735,Baseball Field
3,Sean's Arcade And Mini Bar,4e9cca65dab4aec571ae9d98,Shays Animal Kingdom,14,No Address,39.018282,-94.478710,4bf58dd8d48988d1e5941735,Dog Run
5,Sean's Arcade And Mini Bar,5f29ce36dada2c4da2b70b12,Mount Carmel Cemetary,1115,No Address,39.023317,-94.467750,4bf58dd8d48988d15c941735,Cemetery


### Sorting by Arcade and Venue Distance

In [14]:
surrounding_venues = surrounding_venues.sort_values(['Arcade', 'Venue_Distance'],ascending = True)
surrounding_venues.head(10)

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
28,Arcade Shoe Repair,4d06676137036dcbdaf21bfb,Courtyard Delicatessen,32,1010 Grand Blvd,39.101554,-94.581418,54f4ba06498e2cf5561da814,Corporate Cafeteria
20,Arcade Shoe Repair,4fd8e282e4b0de27c90b4166,Charisse,37,1006 Walnut St,39.101625,-94.582211,4bf58dd8d48988d10c941735,French Restaurant
27,Arcade Shoe Repair,4df8bc09a809df9984ea7cde,Harmony,40,911 Main St,39.101510,-94.582260,4bf58dd8d48988d124941735,Office
26,Arcade Shoe Repair,51b9d3c2498e51f3058aafbf,U.S. Department of Education (ED),56,1010 Walnut St,39.101503,-94.582445,4bf58dd8d48988d124941735,Office
30,Arcade Shoe Repair,502d3f88e4b05f26ae8380f8,KPMG,56,1000 Walnut St Ste 1000,39.101503,-94.582445,4bf58dd8d48988d124941735,Office
29,Arcade Shoe Repair,4c72c5787121a1cd046a63d1,FedEx Office Print & Ship Center,70,"1111 Main St, Suite 111",39.100997,-94.582217,52f2ab2ebcbc57f1066b8b1f,Shipping Store
34,Arcade Shoe Repair,4bed86856e8c20a1bc7d7061,In & Out,85,1007 Main St,39.102059,-94.582522,4d954b0ea243a5684a65b473,Convenience Store
31,Arcade Shoe Repair,4dc03758fa8cc78a8624e726,Mama Tio's,94,1111 Main St,39.100947,-94.582577,4bf58dd8d48988d1c1941735,Mexican Restaurant
22,Arcade Shoe Repair,5c8b1c800d8a0f002c113a68,Spokes Cafe & Cyclery,99,929 Walnut St,39.102437,-94.581747,4bf58dd8d48988d16d941735,Café
33,Arcade Shoe Repair,4e20894bd164740631fcf388,Commerce Bank,102,1001 Main St,39.102431,-94.582076,4bf58dd8d48988d10a951735,Bank


In [15]:
surrounding_venues.to_csv('surrounding_venues_sorted.csv')

### Ensuring Unique Venues

In [16]:
surrounding_sorted = pd.read_csv('surrounding_venues_sorted.csv')
surrounding_sorted = surrounding_sorted.drop('Unnamed: 0', axis= 1)
print(len(surrounding_sorted))
surrounding_sorted.head(20)

131


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Shoe Repair,4d06676137036dcbdaf21bfb,Courtyard Delicatessen,32,1010 Grand Blvd,39.101554,-94.581418,54f4ba06498e2cf5561da814,Corporate Cafeteria
1,Arcade Shoe Repair,4fd8e282e4b0de27c90b4166,Charisse,37,1006 Walnut St,39.101625,-94.582211,4bf58dd8d48988d10c941735,French Restaurant
2,Arcade Shoe Repair,4df8bc09a809df9984ea7cde,Harmony,40,911 Main St,39.101510,-94.582260,4bf58dd8d48988d124941735,Office
3,Arcade Shoe Repair,51b9d3c2498e51f3058aafbf,U.S. Department of Education (ED),56,1010 Walnut St,39.101503,-94.582445,4bf58dd8d48988d124941735,Office
4,Arcade Shoe Repair,502d3f88e4b05f26ae8380f8,KPMG,56,1000 Walnut St Ste 1000,39.101503,-94.582445,4bf58dd8d48988d124941735,Office
5,Arcade Shoe Repair,4c72c5787121a1cd046a63d1,FedEx Office Print & Ship Center,70,"1111 Main St, Suite 111",39.100997,-94.582217,52f2ab2ebcbc57f1066b8b1f,Shipping Store
6,Arcade Shoe Repair,4bed86856e8c20a1bc7d7061,In & Out,85,1007 Main St,39.102059,-94.582522,4d954b0ea243a5684a65b473,Convenience Store
7,Arcade Shoe Repair,4dc03758fa8cc78a8624e726,Mama Tio's,94,1111 Main St,39.100947,-94.582577,4bf58dd8d48988d1c1941735,Mexican Restaurant
8,Arcade Shoe Repair,5c8b1c800d8a0f002c113a68,Spokes Cafe & Cyclery,99,929 Walnut St,39.102437,-94.581747,4bf58dd8d48988d16d941735,Café
9,Arcade Shoe Repair,4e20894bd164740631fcf388,Commerce Bank,102,1001 Main St,39.102431,-94.582076,4bf58dd8d48988d10a951735,Bank


### Pulling the Ratings

In [17]:
count = -1
rating_list = list()
for ident in surrounding_sorted['Venue_ID']:
    count = count +1
    venue_id = surrounding_sorted['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 7.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.0,
 'Not Rated',
 5.2,
 'Not Rated',
 7.5,
 'Not Rated',
 'Not Rated',
 8.5,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.9,
 'Not Rated',
 'Not Rated',
 6.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.7,
 'Not Rated',
 8.3,
 'Not Rated',
 'Not Rated',
 6.5,
 5.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.9,
 7.7,
 'Not Rated',
 'Not Rated',
 7.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.0,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.3,
 'Not Rated',
 'No

In [18]:
surrounding_final = surrounding_sorted.assign(Venue_Rating= rating_list)
surrounding_final.head()

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Arcade Shoe Repair,4d06676137036dcbdaf21bfb,Courtyard Delicatessen,32,1010 Grand Blvd,39.101554,-94.581418,54f4ba06498e2cf5561da814,Corporate Cafeteria,Not Rated
1,Arcade Shoe Repair,4fd8e282e4b0de27c90b4166,Charisse,37,1006 Walnut St,39.101625,-94.582211,4bf58dd8d48988d10c941735,French Restaurant,7.6
2,Arcade Shoe Repair,4df8bc09a809df9984ea7cde,Harmony,40,911 Main St,39.101510,-94.582260,4bf58dd8d48988d124941735,Office,Not Rated
3,Arcade Shoe Repair,51b9d3c2498e51f3058aafbf,U.S. Department of Education (ED),56,1010 Walnut St,39.101503,-94.582445,4bf58dd8d48988d124941735,Office,Not Rated
4,Arcade Shoe Repair,502d3f88e4b05f26ae8380f8,KPMG,56,1000 Walnut St Ste 1000,39.101503,-94.582445,4bf58dd8d48988d124941735,Office,Not Rated


In [19]:
counts = dict()
for category in surrounding_final['Category_Name']:
    counts[category]= counts.get(category, 0) + 1

In [20]:
counts

{'Corporate Cafeteria': 1,
 'French Restaurant': 2,
 'Office': 12,
 'Shipping Store': 2,
 'Convenience Store': 4,
 'Mexican Restaurant': 2,
 'Café': 2,
 'Bank': 4,
 'Residential Building (Apartment / Condo)': 7,
 'Bus Stop': 2,
 'Mediterranean Restaurant': 1,
 "Dentist's Office": 3,
 'Building': 10,
 'Deli / Bodega': 3,
 'Hotel': 2,
 'Beer Garden': 1,
 'Shrine': 1,
 'Hospital': 1,
 'Church': 5,
 'Park': 2,
 'Gym': 1,
 'Bar': 2,
 'School': 1,
 'Liquor Store': 1,
 'Laundry Service': 1,
 'Pizza Place': 2,
 'Scenic Lookout': 1,
 'Other Great Outdoors': 1,
 'Speakeasy': 1,
 'Music Venue': 1,
 'Dive Bar': 2,
 'Steakhouse': 1,
 'Social Club': 1,
 'Sandwich Place': 2,
 'Massage Studio': 1,
 'Conference Room': 2,
 'Art Gallery': 1,
 'Coffee Shop': 1,
 'Parking': 1,
 'Fountain': 1,
 'Financial or Legal Service': 1,
 'Real Estate Office': 1,
 'Theme Park': 14,
 'Theme Park Ride / Attraction': 1,
 'Boutique': 1,
 'American Restaurant': 1,
 'BBQ Joint': 1,
 'Water Park': 1,
 'Event Space': 1,
 'Gym

In [21]:
surrounding_final.to_csv('surrounding_final.csv')